# HRM (Hierarchical Reasoning Model) Trading Training on Colab/Kaggle

This notebook converts the HRM training pipeline to run on Google Colab or Kaggle notebooks with automatic TPU/GPU/CPU detection.

## Overview
- **HRM Architecture**: Brain-inspired hierarchical model with 27M parameters
- **Training Approach**: Deep supervision with adaptive computation time
- **Data Processing**: Automated pipeline from raw to training-ready data
- **Hardware Support**: Automatic TPU/GPU/CPU detection with parallel training

Based on the research paper: *Hierarchical Reasoning Model* by Guan Wang et al.

## 1. Setup Environment and Clone Repository

Clone the private GitHub repository using the service token.

In [ ]:
import os
import sys
import subprocess

# Detect runtime environment
def is_colab():
    return "COLAB_GPU" in os.environ

def is_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

def check_tpu_availability():
    try:
        import requests
        response = requests.get(
            'http://metadata.google.internal/computeMetadata/v1/instance/attributes/accelerator-type',
            headers={'Metadata-Flavor': 'Google'},
            timeout=5
        )
        return 'tpu' in response.text.lower()
    except:
        return False

in_cloud_env = is_colab() or is_kaggle()
is_tpu_environment = check_tpu_availability() if in_cloud_env else False

print(f"🔍 Running in Colab: {is_colab()} | Kaggle: {is_kaggle()} | TPU: {is_tpu_environment}")

# Repo details
repo_url = "https://{personal_access_token}@github.com/marshaltudu14/AlgoTrading.git"
repo_path = "/content/AlgoTrading"

if in_cloud_env or is_tpu_environment:
    # Clone repository only if running in cloud
    if not os.path.exists(repo_path):
        print("📥 Cloning AlgoTrading repository...")
        result = subprocess.run(
            ["git", "clone", repo_url, repo_path],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("✅ Repository cloned successfully!")
        else:
            print(f"❌ Failed to clone repository: {result.stderr}")
            raise Exception("Repository clone failed")
    else:
        print("✅ Repository already exists")

    # Change to repository directory
    os.chdir(repo_path)
    sys.path.append(repo_path)

    print(f"📁 Current directory: {os.getcwd()}")
    print(f"📋 Repository contents:")
    for item in sorted(os.listdir('.')):
        print(f"  {item}")
else:
    print("⏭️ Local environment detected. Skipping repository cloning.")

🔍 Running in Colab: False | Kaggle: False | TPU: False
⏭️ Local environment detected. Skipping repository cloning.


## 2. Install Dependencies

Install all required packages including TPU support for Google Colab.

In [5]:
import os
import sys

# Detect runtime environment
def is_colab():
    return "COLAB_GPU" in os.environ

def is_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

def check_tpu_availability():
    try:
        import requests
        response = requests.get(
            'http://metadata.google.internal/computeMetadata/v1/instance/attributes/accelerator-type',
            headers={'Metadata-Flavor': 'Google'},
            timeout=5
        )
        return 'tpu' in response.text.lower()
    except:
        return False

in_cloud_env = is_colab() or is_kaggle()
is_tpu_environment = check_tpu_availability() if in_cloud_env else False

print(f"🔍 Running in Colab: {is_colab()} | Kaggle: {is_kaggle()} | TPU: {is_tpu_environment}")

# Install only if in cloud (Colab/Kaggle/TPU)
if in_cloud_env or is_tpu_environment:
    print("📦 Installing base requirements...")
    !pip install -r requirements.txt

    if is_tpu_environment:
        print("🚀 Installing TPU support...")
        !pip install torch_xla cloud-tpu-client

    print("📊 Installing visualization libraries...")
    !pip install ipywidgets plotly kaleido

    print("✅ All dependencies installed successfully!")
else:
    print("⏭️ Local environment detected. Skipping installation.")

🔍 Running in Colab: False | Kaggle: False | TPU: False
⏭️ Local environment detected. Skipping installation.


## 3. Device Detection and Setup

Automatically detect and configure the best available device (TPU > GPU > CPU).

In [6]:
import torch
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Import device manager after installation
from src.utils.device_manager import get_device_manager

# Initialize device manager
device_manager = get_device_manager()
device = device_manager.get_device()
device_type = device_manager.get_device_type()

# Print detailed device information
device_manager.print_device_summary()

# Get optimal batch size recommendation
recommended_batch_size = device_manager.get_batch_size_recommendation(32)
print(f"📏 Recommended batch size: {recommended_batch_size}")

# Store device info for later use
DEVICE = device
DEVICE_TYPE = device_type
BATCH_SIZE = recommended_batch_size

print(f"\n🎯 Training will use: {DEVICE} with batch size {BATCH_SIZE}")

2025-08-31 15:37:40,274 - INFO - 💻 Using CPU for training (TPU/GPU not available)



🔥 DEVICE CONFIGURATION SUMMARY
Selected Device: cpu
Device Type: CPU
CPU Cores: 4
CPU Threads: 4

📏 Recommended batch size: 16

🎯 Training will use: cpu with batch size 16


## 4. Data Processing Pipeline

Run the data processing pipeline to convert raw data into training-ready features.

In [7]:
import sys
import os
from pathlib import Path

# Import data processing pipeline
from src.data_processing.pipeline import DataProcessingPipeline

print("🔄 Initializing Data Processing Pipeline...")
print("=" * 80)

# Initialize pipeline
pipeline = DataProcessingPipeline()

# Check if raw data exists
raw_data_path = Path('data/raw')
final_data_path = Path('data/final')

print(f"📂 Raw data path: {raw_data_path}")
print(f"📂 Final data path: {final_data_path}")

# List raw data files
if raw_data_path.exists():
    raw_files = list(raw_data_path.glob("*.csv"))
    print(f"📄 Found {len(raw_files)} raw data files:")
    for file in raw_files[:5]:  # Show first 5 files
        print(f"  - {file.name}")
    if len(raw_files) > 5:
        print(f"  ... and {len(raw_files) - 5} more files")
else:
    print("⚠️ No raw data directory found")

# Run feature generation pipeline
print("\n🚀 Running Feature Generation Pipeline...")
print("=" * 80)

try:
    result = pipeline.run_complete_pipeline(
        input_dir=str(raw_data_path),
        output_dir=str(final_data_path)
    )
    
    if result['success']:
        print("\n✅ Data Processing Pipeline Completed Successfully!")
        print("=" * 80)
        print(f"📊 Summary:")
        print(f"  • Total files processed: {result.get('total_files_processed', 'Unknown')}")
        print(f"  • Total rows processed: {result.get('total_rows_processed', 0):,}")
        print(f"  • Processing time: {result.get('total_time_formatted', 'Unknown')}")
        print(f"  • Output directory: {result.get('output_directory', 'Unknown')}")
        
        # List final data files
        if final_data_path.exists():
            final_files = list(final_data_path.glob("features_*.csv"))
            print(f"\n📈 Generated {len(final_files)} feature files:")
            for file in final_files:
                print(f"  - {file.name}")
    else:
        print(f"❌ Data processing failed: {result.get('error', 'Unknown error')}")
        raise Exception("Data processing pipeline failed")
        
except Exception as e:
    print(f"❌ Pipeline execution failed: {str(e)}")
    # Continue anyway if some data exists
    if final_data_path.exists() and list(final_data_path.glob("features_*.csv")):
        print("⚠️ Using existing processed data files")
    else:
        raise

print("\n🎯 Data processing complete. Ready for training!")

2025-08-31 15:37:41,097 - INFO - DataProcessingPipeline initialized
2025-08-31 15:37:41,101 - INFO - ================================================================================
2025-08-31 15:37:41,101 - INFO - STARTING COMPLETE DATA PROCESSING PIPELINE
2025-08-31 15:37:41,106 - INFO - ================================================================================
2025-08-31 15:37:41,106 - INFO - STEP 1: FEATURE GENERATION
2025-08-31 15:37:41,106 - INFO - ----------------------------------------
2025-08-31 15:37:41,106 - INFO - Running feature generator...


🔄 Initializing Data Processing Pipeline...
📂 Raw data path: data\raw
📂 Final data path: data\final
📄 Found 60 raw data files:
  - Bankex_10.csv
  - Bankex_120.csv
  - Bankex_15.csv
  - Bankex_180.csv
  - Bankex_2.csv
  ... and 55 more files

🚀 Running Feature Generation Pipeline...


c:\Users\iamma\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas_ta\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
2025-08-31 15:37:41,744 - INFO - Found 60 CSV files to process
2025-08-31 15:37:41,770 - INFO - Dropped 'volume' column from Bankex_10.csv
2025-08-31 15:37:41,786 - INFO - Loaded 10127 rows from Bankex_10.csv
2025-08-31 15:37:41,786 - INFO - Processing in-memory DataFrame with 10127 rows...
2025-08-31 15:37:45,426 - INFO - ✅ Enhanced datetime processing: epoch as feature, readable as index
2025-08-31 15:37:55,218 - INFO - Removed 199 rows with NaN values. Final dataset: 9928 rows
2025-08-31 15:37:55,221 - INFO - Generated 48 features for DataFrame
2025-08-31 15:37:55,768 - INFO - Proces

KeyboardInterrupt: 

## 5. HRM Training Configuration

Configure the HRM training parameters and initialize the trainer.

In [12]:
import yaml
import torch
from pathlib import Path

# Load HRM configuration
with open('config/hrm_config.yaml', 'r') as f:
    hrm_config = yaml.safe_load(f)

print("⚙️ HRM Configuration Loaded:")
print("=" * 50)
print(f"🧠 Model Architecture:")
print(f"  • Hidden Dimension: {hrm_config['model_architecture']['hidden_dim']}")
print(f"  • H-Module Layers: {hrm_config['model_architecture']['H_layers']}")
print(f"  • L-Module Layers: {hrm_config['model_architecture']['L_layers']}")
print(f"  • Attention Heads: {hrm_config['model_architecture']['num_heads']}")
print(f"  • H-Cycles: {hrm_config['model_architecture']['H_cycles']}")
print(f"  • L-Cycles: {hrm_config['model_architecture']['L_cycles']}")

print(f"\n🔄 Hierarchical Processing:")
print(f"  • H-Module Lookback: {hrm_config['hierarchical_processing']['high_level_lookback']} candles")
print(f"  • L-Module Lookback: {hrm_config['hierarchical_processing']['low_level_lookback']} candles")

print(f"\n🎓 Training Configuration:")
print(f"  • Base Learning Rate: {hrm_config['training']['learning_rates']['base_lr']}")
print(f"  • Strategic LR: {hrm_config['training']['learning_rates']['strategic_lr']}")
print(f"  • Tactical LR: {hrm_config['training']['learning_rates']['tactical_lr']}")
print(f"  • Optimizer: {hrm_config['training']['optimizer']}")

# Training parameters for notebook environment
TRAINING_PARAMS = {
    'epochs': 100,  # Start with 100 epochs, can be increased
    'save_frequency': 25,  # Save checkpoint every 25 epochs
    'log_frequency': 5,   # Log progress every 5 epochs
    'validation_frequency': 10,  # Validate every 10 epochs
    'debug_mode': True,  # Set to True for detailed step-by-step logging
    'config_path': 'config/hrm_config.yaml',
    'data_path': 'data/final'
}

print(f"\n🎯 Notebook Training Parameters:")
for key, value in TRAINING_PARAMS.items():
    print(f"  • {key}: {value}")

print(f"\n📱 Device Configuration:")
print(f"  • Device: {DEVICE}")
print(f"  • Device Type: {DEVICE_TYPE}")
print(f"  • Batch Size: {BATCH_SIZE}")

⚙️ HRM Configuration Loaded:
🧠 Model Architecture:
  • Hidden Dimension: 512
  • H-Module Layers: 8
  • L-Module Layers: 6
  • Attention Heads: 16
  • H-Cycles: 2
  • L-Cycles: 6

🔄 Hierarchical Processing:
  • H-Module Lookback: 100 candles
  • L-Module Lookback: 15 candles

🎓 Training Configuration:
  • Base Learning Rate: 0.0001
  • Strategic LR: 5e-05
  • Tactical LR: 0.0001
  • Optimizer: AdamW

🎯 Notebook Training Parameters:
  • epochs: 100
  • save_frequency: 25
  • log_frequency: 5
  • validation_frequency: 10
  • debug_mode: True
  • config_path: config/hrm_config.yaml
  • data_path: data/final

📱 Device Configuration:
  • Device: cpu
  • Device Type: cpu
  • Batch Size: 16


## 6. Initialize HRM Training Pipeline

Initialize the HRM training pipeline with automatic device detection and parallel training support.

In [13]:
from run_training import HRMTrainingPipeline
import logging

# Setup enhanced logging for notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  # Only console output for notebook
    ]
)
logger = logging.getLogger(__name__)

print("🚀 Initializing HRM Training Pipeline...")
print("=" * 80)

try:
    # Initialize training pipeline with device detection
    pipeline = HRMTrainingPipeline(
        config_path=TRAINING_PARAMS['config_path'],
        data_path=TRAINING_PARAMS['data_path'],
        device=str(DEVICE),
        debug_mode=TRAINING_PARAMS['debug_mode']
    )
    
    print("✅ Training pipeline initialized successfully!")
    print(f"📊 Available instruments: {len(pipeline.available_instruments)}")
    
    # List available instruments
    if pipeline.available_instruments:
        print("\n📈 Available Training Instruments:")
        for i, instrument in enumerate(pipeline.available_instruments, 1):
            print(f"  {i}. {instrument}")
        
        # Auto-select first instrument for training
        selected_instrument = pipeline.available_instruments[0]
        print(f"\n🎯 Auto-selected instrument: {selected_instrument}")
    else:
        raise ValueError("No instruments available for training")
        
except Exception as e:
    print(f"❌ Failed to initialize training pipeline: {str(e)}")
    raise

print("\n🔥 Ready to start HRM training!")

2025-08-31 15:40:26,638 - INFO - Loaded configuration from config/hrm_config.yaml
2025-08-31 15:40:26,642 - INFO - Validating 2 instrument symbols: ['Bankex', 'Bank_Nifty']
2025-08-31 15:40:26,643 - INFO - Found valid instrument: Bankex (ID: 2)
2025-08-31 15:40:26,644 - INFO - Found valid instrument: Bank_Nifty (ID: 0)
2025-08-31 15:40:26,644 - INFO - Found 2 valid instruments for training
2025-08-31 15:40:26,646 - INFO - HRM Training Pipeline initialized on cpu
2025-08-31 15:40:26,648 - INFO - Debug mode: ON - detailed step-by-step logging


🚀 Initializing HRM Training Pipeline...
✅ Training pipeline initialized successfully!
📊 Available instruments: 2

📈 Available Training Instruments:
  1. Bankex
  2. Bank_Nifty

🎯 Auto-selected instrument: Bankex

🔥 Ready to start HRM training!


## 7. Training Loss Visualization Setup

Setup real-time visualization for training metrics and loss curves.

In [14]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from IPython.display import display, clear_output
import threading
import time

class HRMTrainingVisualizer:
    """Real-time training visualization for Jupyter notebooks"""
    
    def __init__(self):
        self.training_history = []
        self.loss_history = []
        self.reward_history = []
        self.best_reward = float('-inf')
        
        # Setup plots
        self.fig = None
        self.setup_plots()
    
    def setup_plots(self):
        """Initialize the plotting framework"""
        plt.style.use('default')
        plt.rcParams['figure.figsize'] = (15, 10)
        
    def update_metrics(self, episode, metrics):
        """Update training metrics"""
        self.training_history.append({
            'episode': episode,
            'total_reward': metrics.get('total_reward', 0),
            'avg_reward': metrics.get('avg_reward', 0),
            'total_loss': metrics.get('total_loss', 0),
            'avg_loss': metrics.get('avg_loss', 0),
            'steps': metrics.get('steps', 0)
        })
        
        self.reward_history.append(metrics.get('avg_reward', 0))
        self.loss_history.append(metrics.get('avg_loss', 0))
        
        if metrics.get('avg_reward', 0) > self.best_reward:
            self.best_reward = metrics.get('avg_reward', 0)
    
    def create_training_plots(self):
        """Create comprehensive training plots"""
        if len(self.training_history) < 2:
            return
        
        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Average Reward per Episode', 'Average Loss per Episode', 
                          'Total Reward Trend', 'Training Progress Summary'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )
        
        episodes = [h['episode'] for h in self.training_history]
        
        # Reward plot
        fig.add_trace(
            go.Scatter(x=episodes, y=self.reward_history, 
                      mode='lines+markers', name='Avg Reward',
                      line=dict(color='green', width=2)),
            row=1, col=1
        )
        
        # Loss plot
        fig.add_trace(
            go.Scatter(x=episodes, y=self.loss_history,
                      mode='lines+markers', name='Avg Loss',
                      line=dict(color='red', width=2)),
            row=1, col=2
        )
        
        # Total reward trend
        total_rewards = [h['total_reward'] for h in self.training_history]
        fig.add_trace(
            go.Scatter(x=episodes, y=total_rewards,
                      mode='lines+markers', name='Total Reward',
                      line=dict(color='blue', width=2)),
            row=2, col=1
        )
        
        # Summary metrics
        if len(self.reward_history) >= 10:
            recent_avg_reward = np.mean(self.reward_history[-10:])
            recent_avg_loss = np.mean(self.loss_history[-10:])
        else:
            recent_avg_reward = np.mean(self.reward_history)
            recent_avg_loss = np.mean(self.loss_history)
        
        # Create summary text
        summary_text = f"""
        📊 Training Summary (Last 10 Episodes)
        • Best Reward: {self.best_reward:.4f}
        • Recent Avg Reward: {recent_avg_reward:.4f}
        • Recent Avg Loss: {recent_avg_loss:.4f}
        • Total Episodes: {len(self.training_history)}
        """
        
        fig.add_annotation(
            text=summary_text,
            xref="paper", yref="paper",
            x=0.75, y=0.3, xanchor='left', yanchor='top',
            showarrow=False,
            font=dict(size=12, family="monospace"),
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="black",
            borderwidth=1
        )
        
        fig.update_layout(
            height=800,
            title_text="HRM Training Progress - Real-time Monitoring",
            showlegend=True
        )
        
        return fig
    
    def display_current_metrics(self):
        """Display current training metrics"""
        if not self.training_history:
            return
            
        latest = self.training_history[-1]
        
        print(f"\n📈 Episode {latest['episode']} Results:")
        print(f"  🎯 Average Reward: {latest['avg_reward']:.4f}")
        print(f"  📉 Average Loss: {latest['avg_loss']:.4f}")
        print(f"  🎮 Steps Completed: {latest['steps']}")
        print(f"  🏆 Best Reward So Far: {self.best_reward:.4f}")
        
        if latest['avg_reward'] == self.best_reward:
            print("  🌟 NEW BEST PERFORMANCE! 🌟")

# Initialize visualizer
visualizer = HRMTrainingVisualizer()
print("📊 Training visualizer initialized!")
print("📈 Real-time plots will be updated during training")

📊 Training visualizer initialized!
📈 Real-time plots will be updated during training


## 8. Start HRM Training

Begin the HRM training process with real-time monitoring and visualization.

In [15]:
import time
from IPython.display import display, clear_output
import ipywidgets as widgets

print("🚀 Starting HRM Training Process...")
print("=" * 80)

# Create training progress widgets
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=TRAINING_PARAMS['epochs'],
    description='Training:',
    bar_style='info',
    style={'bar_color': 'green'},
    orientation='horizontal'
)

status_text = widgets.HTML(value="<b>Initializing training...</b>")
metrics_text = widgets.HTML(value="")

display(widgets.VBox([progress_bar, status_text, metrics_text]))

# Custom training loop with visualization
class HRMTrainingMonitor:
    def __init__(self, pipeline, visualizer):
        self.pipeline = pipeline
        self.visualizer = visualizer
        self.start_time = time.time()
        
    def run_training_with_monitoring(self, epochs, instrument_symbol):
        """Run training with real-time monitoring"""
        
        # Setup training 
        self.pipeline.setup_training(instrument_symbol)
        
        status_text.value = f"<b>🎯 Training on {instrument_symbol} for {epochs} epochs</b>"
        
        training_history = []
        
        for episode in range(epochs):
            episode_start_time = time.time()
            
            # Run single episode
            try:
                metrics = self.pipeline.trainer.train_episode()
                training_history.append(metrics)
                
                # Update visualizer
                self.visualizer.update_metrics(episode + 1, metrics)
                
                # Advance to next episode
                self.pipeline.trainer._advance_to_next_episode()
                
                # Update learning rate
                if self.pipeline.trainer.scheduler:
                    self.pipeline.trainer.scheduler.step()
                
                # Update progress
                progress_bar.value = episode + 1
                
                # Calculate timing
                episode_time = time.time() - episode_start_time
                elapsed_time = time.time() - self.start_time
                estimated_total = elapsed_time * epochs / (episode + 1)
                remaining_time = estimated_total - elapsed_time
                
                # Update status
                status_text.value = f"""
                <b>📈 Episode {episode + 1}/{epochs} completed</b><br>
                ⏱️ Episode Time: {episode_time:.1f}s<br>
                🕐 Elapsed: {elapsed_time/60:.1f}min | Remaining: {remaining_time/60:.1f}min
                """
                
                # Update metrics display
                recent_rewards = [h['avg_reward'] for h in training_history[-5:]]
                recent_losses = [h['avg_loss'] for h in training_history[-5:]]
                
                metrics_text.value = f"""
                <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
                <b>📊 Current Metrics:</b><br>
                🎯 Avg Reward: <span style="color: green;"><b>{metrics['avg_reward']:.4f}</b></span><br>
                📉 Avg Loss: <span style="color: red;"><b>{metrics['avg_loss']:.4f}</b></span><br>
                🎮 Steps: {metrics['steps']}<br>
                🏆 Best Reward: <span style="color: blue;"><b>{self.visualizer.best_reward:.4f}</b></span><br>
                📈 Recent Avg Reward (last 5): {np.mean(recent_rewards):.4f}<br>
                📉 Recent Avg Loss (last 5): {np.mean(recent_losses):.4f}
                </div>
                """
                
                # Save checkpoints
                if (episode + 1) % TRAINING_PARAMS['save_frequency'] == 0:
                    self.pipeline.trainer._save_latest_checkpoint(episode)
                    
                # Check for best model
                if metrics['avg_reward'] > self.pipeline.trainer.best_performance:
                    self.pipeline.trainer.best_performance = metrics['avg_reward']
                    self.pipeline.trainer._save_best_model()
                    
                # Update plots every 10 episodes
                if (episode + 1) % 10 == 0:
                    clear_output(wait=True)
                    display(widgets.VBox([progress_bar, status_text, metrics_text]))
                    
                    # Show training plots
                    fig = self.visualizer.create_training_plots()
                    if fig:
                        fig.show()
                        
            except KeyboardInterrupt:
                print("\n⚠️ Training interrupted by user")
                break
            except Exception as e:
                print(f"❌ Error in episode {episode + 1}: {str(e)}")
                continue
        
        return training_history

# Initialize training monitor
monitor = HRMTrainingMonitor(pipeline, visualizer)

# Start training
try:
    training_results = monitor.run_training_with_monitoring(
        epochs=TRAINING_PARAMS['epochs'],
        instrument_symbol=selected_instrument
    )
    
    print("\n✅ Training completed successfully!")
    
except Exception as e:
    print(f"❌ Training failed: {str(e)}")
    import traceback
    traceback.print_exc()

🚀 Starting HRM Training Process...


2025-08-31 15:40:39,733 - INFO - setup_training called with: Bankex, timeframe: 5
2025-08-31 15:40:39,734 - INFO - Available instruments: ['Bankex', 'Bank_Nifty']
2025-08-31 15:40:39,737 - WARNING - Exact file features_Bankex_5.csv not found. Available files:
2025-08-31 15:40:39,738 - WARNING -   features_Bankex_10.csv
2025-08-31 15:40:39,739 - WARNING -   features_Bankex_120.csv
2025-08-31 15:40:39,740 - WARNING -   features_Bankex_15.csv
2025-08-31 15:40:39,741 - WARNING -   features_Bankex_180.csv
2025-08-31 15:40:39,741 - INFO - Using available file: features_Bankex_10.csv -> symbol: Bankex_10
2025-08-31 15:40:39,793 - INFO - Trainer initialized for Bankex


❌ Error in episode 1: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 2: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 3: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 4: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 5: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 6: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 7: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 8: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 9: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 10: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 11: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 12: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 13: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 14: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 15: 'HRMTrainer' object has no attribute 'env'
❌ Error in episode 

## 9. Training Results Analysis

Analyze the training results and evaluate model performance.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

print("📊 Analyzing Training Results...")
print("=" * 80)

# Final training visualization
if visualizer.training_history:
    print("📈 Generating Final Training Report...")
    
    # Create comprehensive plots
    final_fig = visualizer.create_training_plots()
    if final_fig:
        final_fig.update_layout(title_text="HRM Training Results - Final Report")
        final_fig.show()
    
    # Statistical analysis
    rewards = visualizer.reward_history
    losses = visualizer.loss_history
    
    print(f"\n📈 Training Statistics:")
    print(f"  🎯 Total Episodes Completed: {len(rewards)}")
    print(f"  🏆 Best Reward Achieved: {visualizer.best_reward:.4f}")
    print(f"  📊 Final Average Reward: {rewards[-1]:.4f}")
    print(f"  📉 Final Average Loss: {losses[-1]:.4f}")
    
    if len(rewards) >= 20:
        print(f"\n📊 Performance Trends:")
        early_reward = np.mean(rewards[:10])
        late_reward = np.mean(rewards[-10:])
        reward_improvement = late_reward - early_reward
        
        early_loss = np.mean(losses[:10])
        late_loss = np.mean(losses[-10:])
        loss_improvement = early_loss - late_loss
        
        print(f"  📈 Reward Improvement: {reward_improvement:.4f} ({reward_improvement/early_reward*100:+.1f}%)")
        print(f"  📉 Loss Improvement: {loss_improvement:.4f} ({loss_improvement/early_loss*100:+.1f}%)")
    
    # Model performance analysis
    print(f"\n🧠 Model Performance Analysis:")
    if len(rewards) > 0:
        reward_variance = np.var(rewards)
        reward_stability = 1.0 / (1.0 + reward_variance) if reward_variance > 0 else 1.0
        print(f"  📊 Reward Variance: {reward_variance:.4f}")
        print(f"  🎯 Training Stability: {reward_stability:.3f}")
        
        # Performance rating
        if visualizer.best_reward > 0.1:
            performance_rating = "🌟 Excellent"
        elif visualizer.best_reward > 0.05:
            performance_rating = "✅ Good"
        elif visualizer.best_reward > 0.0:
            performance_rating = "⚠️ Fair"
        else:
            performance_rating = "❌ Poor"
            
        print(f"  🏆 Overall Performance: {performance_rating}")
    
    # Save training summary
    summary_data = {
        'episode': list(range(1, len(rewards) + 1)),
        'avg_reward': rewards,
        'avg_loss': losses,
        'best_reward_so_far': [max(rewards[:i+1]) for i in range(len(rewards))]
    }
    
    df_summary = pd.DataFrame(summary_data)
    
    # Save to CSV
    results_dir = Path("training_results")
    results_dir.mkdir(exist_ok=True)
    
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    csv_path = results_dir / f"hrm_training_summary_{timestamp}.csv"
    df_summary.to_csv(csv_path, index=False)
    
    print(f"\n💾 Training summary saved to: {csv_path}")
    
    # Display final data sample
    print(f"\n📋 Training Summary (Last 10 Episodes):")
    print(df_summary.tail(10).to_string(index=False))

else:
    print("⚠️ No training history available")

print("\n🎉 Training analysis complete!")

## 10. Model Evaluation and Testing

Evaluate the trained HRM model on test data.

In [ ]:
print("🔬 Evaluating Trained HRM Model...")
print("=" * 80)

try:
    # Run model evaluation
    if hasattr(pipeline, 'trainer') and pipeline.trainer.model is not None:
        print("🧪 Running model evaluation on test episodes...")
        
        eval_results = pipeline.trainer.evaluate(
            episodes=20,  # Evaluate on 20 test episodes
            symbol=selected_instrument
        )
        
        print("\n📊 Model Evaluation Results:")
        print("=" * 50)
        
        for metric, value in eval_results.items():
            if isinstance(value, float):
                print(f"  📈 {metric.replace('_', ' ').title()}: {value:.4f}")
            else:
                print(f"  📈 {metric.replace('_', ' ').title()}: {value}")
        
        # Model summary
        print("\n🧠 HRM Model Architecture Summary:")
        print("=" * 50)
        
        model_summary = pipeline.trainer.model.get_model_summary()
        
        print(f"  🔢 Total Parameters: {model_summary['total_parameters']:,}")
        print(f"  🎓 Trainable Parameters: {model_summary['trainable_parameters']:,}")
        print(f"  🧠 H-Module Parameters: {model_summary['h_module_parameters']:,}")
        print(f"  ⚡ L-Module Parameters: {model_summary['l_module_parameters']:,}")
        print(f"  🔄 ACT Module Parameters: {model_summary['act_module_parameters']:,}")
        print(f"  📚 Deep Supervision Parameters: {model_summary['deep_supervision_parameters']:,}")
        print(f"  📊 Hidden Dimension: {model_summary['hidden_dimension']}")
        print(f"  🔄 H-Cycles: {model_summary['H_cycles']} | L-Cycles: {model_summary['L_cycles']}")
        print(f"  👁️ H-Lookback: {model_summary['h_lookback_window']} | L-Lookback: {model_summary['l_lookback_window']}")
        print(f"  💻 Device: {model_summary['device']}")
        
        # Checkpoint validation
        print("\n🔍 Validating Model Checkpoints...")
        if pipeline.validate_checkpoints():
            print("✅ Model checkpoints are valid and ready for deployment")
            
            # List saved models
            model_dir = Path("models/hrm")
            checkpoint_dir = Path("checkpoints/hrm")
            
            if model_dir.exists():
                model_files = list(model_dir.glob("*.pt"))
                print(f"\n📁 Saved Models ({len(model_files)}):")
                for model_file in model_files:
                    print(f"  💾 {model_file.name}")
            
            if checkpoint_dir.exists():
                checkpoint_files = list(checkpoint_dir.glob("*.pt"))
                print(f"\n📁 Training Checkpoints ({len(checkpoint_files)}):")
                for checkpoint_file in checkpoint_files:
                    print(f"  💾 {checkpoint_file.name}")
        else:
            print("⚠️ Checkpoint validation failed")
        
    else:
        print("⚠️ No trained model available for evaluation")
        
except Exception as e:
    print(f"❌ Evaluation failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("\n🎯 Model evaluation complete!")

## 11. Export Results and Model

Export training results, model files, and create deployment package.

In [ ]:
import shutil
import json
from datetime import datetime

print("📦 Creating Export Package...")
print("=" * 80)

# Create export directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
export_dir = Path(f"hrm_export_{timestamp}")
export_dir.mkdir(exist_ok=True)

print(f"📁 Export directory: {export_dir}")

try:
    # 1. Copy model files
    model_dir = Path("models/hrm")
    if model_dir.exists():
        export_models_dir = export_dir / "models"
        shutil.copytree(model_dir, export_models_dir)
        print(f"✅ Models exported to: {export_models_dir}")
    
    # 2. Copy checkpoints
    checkpoint_dir = Path("checkpoints/hrm")
    if checkpoint_dir.exists():
        export_checkpoints_dir = export_dir / "checkpoints"
        shutil.copytree(checkpoint_dir, export_checkpoints_dir)
        print(f"✅ Checkpoints exported to: {export_checkpoints_dir}")
    
    # 3. Copy training results
    results_dir = Path("training_results")
    if results_dir.exists():
        export_results_dir = export_dir / "training_results"
        shutil.copytree(results_dir, export_results_dir)
        print(f"✅ Training results exported to: {export_results_dir}")
    
    # 4. Copy configuration
    config_files = ['config/hrm_config.yaml']
    export_config_dir = export_dir / "config"
    export_config_dir.mkdir(exist_ok=True)
    
    for config_file in config_files:
        if Path(config_file).exists():
            shutil.copy2(config_file, export_config_dir)
            print(f"✅ Config exported: {config_file}")
    
    # 5. Create training summary report
    summary_report = {
        "training_info": {
            "timestamp": timestamp,
            "device_used": str(DEVICE),
            "device_type": DEVICE_TYPE,
            "batch_size": BATCH_SIZE,
            "epochs_completed": len(visualizer.training_history) if visualizer.training_history else 0,
            "selected_instrument": selected_instrument if 'selected_instrument' in locals() else "Unknown"
        },
        "training_parameters": TRAINING_PARAMS,
        "performance_summary": {
            "best_reward": float(visualizer.best_reward) if visualizer.training_history else 0.0,
            "final_reward": float(visualizer.reward_history[-1]) if visualizer.reward_history else 0.0,
            "final_loss": float(visualizer.loss_history[-1]) if visualizer.loss_history else 0.0,
            "total_episodes": len(visualizer.training_history) if visualizer.training_history else 0
        }
    }
    
    # Add evaluation results if available
    if 'eval_results' in locals():
        summary_report["evaluation_results"] = {
            k: float(v) if isinstance(v, (int, float)) else v 
            for k, v in eval_results.items()
        }
    
    # Save summary report
    summary_file = export_dir / "training_summary.json"
    with open(summary_file, 'w') as f:
        json.dump(summary_report, f, indent=2)
    print(f"✅ Summary report saved: {summary_file}")
    
    # 6. Create README for export
    readme_content = f"""
# HRM Trading Model Export Package

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Training Device:** {DEVICE_TYPE.upper()}
**Instrument:** {selected_instrument if 'selected_instrument' in locals() else 'Unknown'}

## Contents

- `models/` - Trained HRM models (best model: hrm_best_model.pt)
- `checkpoints/` - Training checkpoints for resuming
- `training_results/` - Training logs and metrics
- `config/` - Model configuration files
- `training_summary.json` - Complete training summary

## Model Architecture

- **Total Parameters:** {model_summary['total_parameters']:,} (≈27M)
- **Architecture:** Hierarchical Reasoning Model (HRM)
- **H-Module Lookback:** {model_summary['h_lookback_window']} candles
- **L-Module Lookback:** {model_summary['l_lookback_window']} candles
- **Hidden Dimension:** {model_summary['hidden_dimension']}

## Performance

- **Best Reward:** {visualizer.best_reward:.4f}
- **Episodes Completed:** {len(visualizer.training_history) if visualizer.training_history else 0}
- **Training Status:** {'Completed' if len(visualizer.training_history) >= TRAINING_PARAMS['epochs'] else 'Partial'}

## Usage

Load the best model:
```python
import torch
model_data = torch.load('models/hrm_best_model.pt')
# Model ready for inference
```

Resume training:
```python
trainer.load_checkpoint('checkpoints/latest_checkpoint.pt')
# Continue training from checkpoint
```
    """
    
    readme_file = export_dir / "README.md"
    with open(readme_file, 'w') as f:
        f.write(readme_content)
    print(f"✅ README created: {readme_file}")
    
    # 7. Create deployment script
    deployment_script = f"""
#!/usr/bin/env python3
"""
HRM Model Deployment Script
Auto-generated from training session: {timestamp}
"""

import torch
import sys
import os
from pathlib import Path

def load_hrm_model(model_path="models/hrm_best_model.pt", device="auto"):
    """Load trained HRM model for inference"""
    
    # Auto-detect device if requested
    if device == "auto":
        if torch.cuda.is_available():
            device = "cuda"
        else:
            device = "cpu"
    
    # Load model
    model_data = torch.load(model_path, map_location=device)
    
    print(f"✅ HRM Model loaded successfully")
    print(f"📊 Performance: {{model_data['performance']:.4f}}")
    print(f"💻 Device: {{device}}")
    print(f"📅 Saved: {{model_data['timestamp']}}")
    
    return model_data

if __name__ == "__main__":
    # Example usage
    model = load_hrm_model()
    print("🚀 HRM model ready for inference!")
    """
    
    deploy_script_file = export_dir / "deploy_model.py"
    with open(deploy_script_file, 'w') as f:
        f.write(deployment_script)
    print(f"✅ Deployment script created: {deploy_script_file}")
    
    # 8. Calculate export size
    def get_dir_size(path):
        total_size = 0
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                total_size += os.path.getsize(filepath)
        return total_size
    
    export_size_bytes = get_dir_size(export_dir)
    export_size_mb = export_size_bytes / (1024 * 1024)
    
    print(f"\n📦 Export Package Summary:")
    print(f"=" * 50)
    print(f"📁 Location: {export_dir}")
    print(f"📏 Size: {export_size_mb:.1f} MB")
    
    # List contents
    print(f"\n📋 Package Contents:")
    for item in sorted(export_dir.iterdir()):
        if item.is_dir():
            item_count = len(list(item.iterdir()))
            print(f"  📁 {item.name}/ ({item_count} items)")
        else:
            item_size_kb = item.stat().st_size / 1024
            print(f"  📄 {item.name} ({item_size_kb:.1f} KB)")
    
    print(f"\n✅ Export package created successfully!")
    print(f"🚀 Ready for deployment or further analysis")
    
except Exception as e:
    print(f"❌ Export failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("\n🎉 Training and export process completed!")

## 12. Training Summary and Next Steps

Final summary of the training process and recommendations for next steps.

In [ ]:
print("🎊 HRM TRAINING COMPLETED!")
print("=" * 80)

# Final summary
total_time = time.time() - monitor.start_time if 'monitor' in locals() else 0

print(f"\n📊 FINAL TRAINING SUMMARY:")
print(f"=" * 50)
print(f"🧠 Model: Hierarchical Reasoning Model (HRM)")
print(f"📈 Instrument: {selected_instrument if 'selected_instrument' in locals() else 'Unknown'}")
print(f"💻 Device: {DEVICE_TYPE.upper()} ({DEVICE})")
print(f"⏱️ Total Training Time: {total_time/3600:.1f} hours")
print(f"🎯 Episodes Completed: {len(visualizer.training_history) if visualizer.training_history else 0}/{TRAINING_PARAMS['epochs']}")

if visualizer.training_history:
    print(f"🏆 Best Performance: {visualizer.best_reward:.4f}")
    print(f"📈 Final Performance: {visualizer.reward_history[-1]:.4f}")
    print(f"📉 Final Loss: {visualizer.loss_history[-1]:.4f}")

# Model specifications
if 'model_summary' in locals():
    print(f"\n🧠 MODEL SPECIFICATIONS:")
    print(f"=" * 50)
    print(f"📊 Total Parameters: {model_summary['total_parameters']:,}")
    print(f"🔄 H-Cycles: {model_summary['H_cycles']} | L-Cycles: {model_summary['L_cycles']}")
    print(f"👁️ H-Lookback: {model_summary['h_lookback_window']} | L-Lookback: {model_summary['l_lookback_window']}")
    print(f"🔢 Hidden Dimension: {model_summary['hidden_dimension']}")

# Performance evaluation
if visualizer.training_history:
    performance_level = "🌟 Excellent" if visualizer.best_reward > 0.1 else \
                       "✅ Good" if visualizer.best_reward > 0.05 else \
                       "⚠️ Fair" if visualizer.best_reward > 0.0 else \
                       "❌ Needs Improvement"
    
    print(f"\n🏆 PERFORMANCE EVALUATION: {performance_level}")

# Next steps recommendations
print(f"\n🚀 RECOMMENDED NEXT STEPS:")
print(f"=" * 50)

if visualizer.best_reward > 0.05:
    print(f"✅ Model shows good performance:")
    print(f"  • Deploy model for live testing")
    print(f"  • Run extended evaluation on more instruments")
    print(f"  • Consider ensemble with other models")
elif visualizer.best_reward > 0.0:
    print(f"⚠️ Model shows moderate performance:")
    print(f"  • Continue training for more epochs")
    print(f"  • Experiment with different hyperparameters")
    print(f"  • Try different instruments or timeframes")
else:
    print(f"❌ Model needs improvement:")
    print(f"  • Check data quality and preprocessing")
    print(f"  • Adjust learning rates or architecture")
    print(f"  • Consider curriculum learning approach")

print(f"\n📁 Export package available at: {export_dir if 'export_dir' in locals() else 'Not created'}")

# Additional recommendations
print(f"\n💡 ADDITIONAL RECOMMENDATIONS:")
print(f"=" * 50)
print(f"📊 • Monitor model performance on different market conditions")
print(f"🔄 • Implement continuous learning for market adaptation")
print(f"⚠️ • Add robust risk management and position sizing")
print(f"📈 • Backtest on historical data before live deployment")
print(f"🏭 • Consider distributed training for larger datasets")

print(f"\n🎯 Training session completed successfully!")
print(f"📚 Refer to the HRM research paper for implementation details")
print(f"🔗 Paper: 'Hierarchical Reasoning Model' by Guan Wang et al.")

print("\n" + "=" * 80)
print("🎉 Thank you for using the HRM Training Notebook!")
print("=" * 80)